ASSIGNMENT -> KITE PNL 

In [30]:
#importing necessary libraries
import pandas as pd
import numpy as np

In [31]:
#loading the data from a csv file into a Dataframe
orders = pd.read_csv("orders.csv")

In [32]:
#Understanding the data -> preview of the data
orders.head()

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
0,18/12/21 10:23,SELL,SBI,CNC,1000/1000,525.25,COMPLETE
1,16/12/21 15:08,BUY,ASHOKLEY,MIS,1000/1000,125.70,COMPLETE
2,16/12/21 15:08,BUY,SBI,CNC,1000/1000,520.80,COMPLETE
3,16/12/21 14:13,BUY,TATAMOTORS,MIS,250/250,490.55,COMPLETE
4,16/12/21 13:54,BUY,ASHOKLEY,MIS,0/1000,127.10,CANCELLED


In [33]:
#Getting a summary of the DataFrame
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Time        14 non-null     object 
 1   Type        14 non-null     object 
 2   Instrument  14 non-null     object 
 3   Product     14 non-null     object 
 4   Qty.        14 non-null     object 
 5   Avg. price  14 non-null     float64
 6   Status      14 non-null     object 
dtypes: float64(1), object(6)
memory usage: 916.0+ bytes


In [34]:
#Cleaning the Data

#1. Checking for Missing Values

orders.isna()

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False


In [35]:
#2. Correcting Data types

orders['Time'] = pd.to_datetime(orders["Time"], format="%d/%m/%y %H:%M")

In [36]:
#Filtering data for "MIS" products and "Completed" trades

filtered_orders = orders[orders["Product"] == 'MIS']
filtered_orders

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
1,2021-12-16 15:08:00,BUY,ASHOKLEY,MIS,1000/1000,125.70,COMPLETE
3,2021-12-16 14:13:00,BUY,TATAMOTORS,MIS,250/250,490.55,COMPLETE
4,2021-12-16 13:54:00,BUY,ASHOKLEY,MIS,0/1000,127.10,CANCELLED
5,2021-12-16 13:21:00,SELL,TATAMOTORS,MIS,250/250,492.10,COMPLETE
6,2021-12-16 12:51:00,BUY,TATAMOTORS,MIS,0/250,490.80,CANCELLED
7,2021-12-16 12:39:00,SELL,ASHOKLEY,MIS,1000/1000,125.96,COMPLETE
8,2021-12-16 12:29:00,BUY,ASHOKLEY,MIS,2000/2000,125.70,COMPLETE
9,2021-12-16 11:22:00,SELL,ASHOKLEY,MIS,2000/2000,125.95,COMPLETE
10,2021-12-16 10:47:00,SELL,ASHOKLEY,MIS,0/2000,124.45,CANCELLED
11,2021-12-16 10:46:00,SELL,ASHOKLEY,MIS,2000/2000,125.95,COMPLETE


In [37]:
# Spliting the 'Qty.' column to extract the executed and total quantities

pd.options.mode.copy_on_write = True

filtered_orders[["Qty Executed", "Qty Total"]] = filtered_orders["Qty."].str.split('/', expand=True)
filtered_orders["Qty Executed"] = filtered_orders["Qty Executed"].astype(int)
filtered_orders["Qty Total"] = filtered_orders["Qty Total"].astype(int)

CLIENT REQUIREMENT:

Create a summary file in Excel that should contain three tables in different Excel sheets:

1) Different types of charges for Individual trade

In [38]:
# Calculating the required Charges

#1. Turnover Calculation
# - Turnover is the total traded value for each trade.
# - It is calculated as: Quantity Executed × Average Price.

filtered_orders["Turnover"] =  filtered_orders["Qty Executed"] * filtered_orders["Avg. price"]

#2. Brokerage Calculation
# - Brokerage is the minimum of ₹20 or 0.03% of the turnover.

filtered_orders["Brokerage"] = filtered_orders["Turnover"].apply(lambda x: min(20, (0.03/100) * x))

#3. STT/CTT Calculation (Securities Transaction Tax / Commodities Transaction Tax)
# - STT/CTT is applicable only for SELL trades at 0.025% of Turnover.

def stt_ctt(record) :
    return round(record["Turnover"] * (0.025/100),2) if record["Type"] == 'SELL' else 0
        
filtered_orders["STT\CTT"] = filtered_orders.apply(stt_ctt, axis=1)

#4.  # Transaction Charges Calculation
# - Transaction charges are 0.00297% of the Turnover.

filtered_orders["Transaction Charges"] = round(filtered_orders["Turnover"] * (0.00297/100),2)

#5. SEBI Charges Calculation
# - SEBI turnover fee is ₹10 per ₹1 crore of Turnover.

filtered_orders["SEBI"] = round(filtered_orders["Turnover"] * (10/10000000) , 2)

#6. GST Calculation
# - GST is 18% on (Brokerage + SEBI Charges + Transaction Charges).  

filtered_orders["GST"] = round((filtered_orders["Brokerage"] + filtered_orders["SEBI"] + filtered_orders["Transaction Charges"]) * (18/100), 2)

#7. Stamp Duty Calculation
# - Stamp duty is 0.003% of Turnover but applicable only for BUY trades.
def stamp_charges(record) : 
    return round(record["Turnover"] * (0.003 / 100), 2) if record["Type"] == 'BUY' else 0
    
filtered_orders["Stamp Charges"] = filtered_orders.apply(stamp_charges, axis=1)


#8. Total Charges Calculation
# - Total charges include all trading fees and taxes.

def total_charges(x) :
    return x["Brokerage"] + x["STT\CTT"] + x["SEBI"] + x["GST"] + x["Stamp Charges"]

filtered_orders["Total Charges"] = filtered_orders.apply(total_charges, axis=1)      

In [39]:
# Table 1: Individual Trade Charges Summary
# - This table contains trade-wise details of all charges applied.

Trade_Charges = filtered_orders[['Instrument', 'Type', 'Qty.', 'Avg. price', 'Status', 'Turnover', 'Brokerage', 
                        'STT\CTT', 'SEBI', 'GST', 'Stamp Charges', 'Total Charges']]




2) Stock wise and Type wise analysis with weighted Avg. price & calculated charges






In [40]:
# Weighted Average Price (WAP) Calculation
# - WAP is calculated by weighting the executed quantity against the average price.
# - Formula: WAP = Σ(Qty Executed × Avg Price) / Σ(Qty Executed)

weighted_avg_price = filtered_orders.groupby(by=["Instrument","Type"]).apply(
               lambda x: (x["Qty Executed"]*x['Avg. price']).sum() / x["Qty Executed"].sum()).reset_index(name="WAP")

weighted_avg_price

,Instrument,Type,WAP
0,ASHOKLEY,BUY,125.670
1,ASHOKLEY,SELL,125.952
2,TATAMOTORS,BUY,490.550
3,TATAMOTORS,SELL,492.100


In [41]:
# Grouping by 'Instrument' and 'Type' (Stock and Trade Type) to aggregate values
# - Summarizes total quantities, prices, charges, and fees for each stock and trade type.

stock_type_analysis  = filtered_orders.groupby(by=["Instrument","Type"], as_index=False).agg({
    'Qty Executed' : 'sum',
    'Avg. price': 'mean', 
    'Turnover': 'sum', 
    'Brokerage': 'sum', 
    'STT\CTT': 'sum', 
    'SEBI': 'sum', 
    'GST': 'sum', 
    'Stamp Charges': 'sum', 
    'Total Charges': 'sum'
})

stock_type_analysis = pd.merge(stock_type_analysis, weighted_avg_price, on=["Instrument","Type"])


In [42]:
# Table 2: Stock-wise and Type-wise Analysis
# - This table provides a detailed breakdown of each stock’s trade activity, charges, and WAP.

stock_type_analysis

,Instrument,Type,Qty Executed,Avg. price,Turnover,Brokerage,STT\CTT,SEBI,GST,Stamp Charges,Total Charges,WAP
0,ASHOKLEY,BUY,5000,125.9500,628350.0,80.0,0.00,0.64,17.86,18.85,117.35,125.670
1,ASHOKLEY,SELL,5000,125.5775,629760.0,60.0,157.45,0.63,14.28,0.00,232.36,125.952
2,TATAMOTORS,BUY,250,490.6750,122637.5,20.0,0.00,0.12,4.28,3.68,28.08,490.550
3,TATAMOTORS,SELL,250,492.1000,123025.0,20.0,30.76,0.12,4.28,0.00,55.16,492.100


3) Overall Summary of each Stocks

In [43]:
# Calculate Gross PnL (Profit and Loss)
# - If the trade is a BUY, the turnover is considered negative (since money is spent).
# - If the trade is a SELL, the turnover is positive (since money is earned).

filtered_orders["Gross PnL"] = np.where(filtered_orders["Type"]=="BUY" ,-filtered_orders["Turnover"],filtered_orders["Turnover"])

In [44]:
# Calculate Net PnL (Net Profit and Loss)
# - Net PnL is calculated as Gross PnL minus the total charges (fees incurred).

filtered_orders["Net PnL"] = filtered_orders["Gross PnL"] - filtered_orders["Total Charges"]

In [45]:
# Create Overall Summary Table
# - This table summarizes the total Gross PnL, Total Charges, Net PnL, and Percentage Charges for each stock.

overall_summary = filtered_orders.groupby("Instrument").agg({
    "Gross PnL" : "sum",
    "Total Charges" : "sum",
    "Net PnL" : "sum"
}).reset_index(False)

In [47]:
# Calculate Percentage of Charges on Gross PnL
# - This metric helps understand how much of the Gross PnL is deducted due to trading charges.

overall_summary["% Charges on Gross PnL"] = round(
    (overall_summary["Total Charges"] / overall_summary["Gross PnL"]) * 100, 2
)
overall_summary

,Instrument,Gross PnL,Total Charges,Net PnL,% Charges on Gross PnL
0,ASHOKLEY,1410.0,349.71,1060.29,24.80
1,TATAMOTORS,387.5,83.24,304.26,21.48


In [49]:
# Saving the Summary Data to an Excel File

with pd.ExcelWriter("Orders_Analysis.xlsx") as writer:
    
    Trade_Charges.to_excel(writer, sheet_name = "Trade Charges", index=False)
    
    stock_type_analysis.to_excel(writer, sheet_name = "Stock Type Analysis", index=False)
    
    overall_summary.to_excel(writer, sheet_name = "Overall Summary", index=False)